# Problem Set One, CR15 ENS de Lyon.
## Course authorities : Karsai / Crespelle / Unicomb

### Students : Mascarade Pierre, Latrille Thibault

Runs on python 2.x

In [1]:
""" IXXI graph library """
from random import random
from random import sample
from collections import deque
import numpy as np

In [2]:
# =========================
#       GRAPH CLASS
# =========================


class Graph(object):
    def __init__(self, graph_dict={}):
        """ initializes a graph object """
        self.__graph_dict = graph_dict

    def vertices(self):
        """ returns the vertices of a graph """
        return sorted(list(self.__graph_dict.keys()))

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def __len__(self):
        return len(self.edges())

    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in
        self.__graph_dict, a key "vertex" with an empty
        list as a value is added to the dictionary.
        Otherwise nothing has to be done.
        """
        if vertex not in self.vertices():
            self.__graph_dict[vertex] = []

    def add_edge(self, edge):
        """ It the edge is not in self.__graph_dict,
        the vertices of the edge are added to each other keys
        The function assumes that edge is of type set, tuple or list;
        (no multiple edges)
        """
        if edge[1] not in self.__graph_dict[edge[0]]:
            self.__graph_dict[edge[0]].append(edge[1])
            self.__graph_dict[edge[1]].append(edge[0])

    def __generate_edges(self):
        """ A static method generating the edges of the
        graph "graph". Edges are represented as sets
        two vertices (no loop)
        """
        edges = []
        for vertex_in in self.vertices():
            for vertex_out in self.__graph_dict[vertex_in]:
                if vertex_in < vertex_out:
                    edges.append((vertex_in, vertex_out))
        return edges

    def vertex_degree(self):
        """ returns a list of sets containing the
        name and degree of each vertex of a graph """
        return [(vertex, len(self.__graph_dict[vertex])) for vertex in self.vertices()]

    def degree_sequence(self):
        """ returns as a non-increasing list sequence of the vertex degrees of a graph """
        return [degree for _, degree in sorted(self.vertex_degree(), key=lambda x: x[1], reverse=True)]

    def erdos_gallai(self, sequence):
        """ for a given degree sequence check if it can be
         realised by a simple graph
         returns a boolean"""
        n = len(sequence)
        if sum(sequence) % 2 == 1:
            return False
        for k in range(1, n + 1):
            if not sum(sequence[:k]) <= k * (k - 1) + sum([min(d, k) for d in sequence[k:n]]):
                return False
        return True

    def find_isolated_vertices(self):
        """ returns the list of zero-degree vertices of a graph """
        return [vertex for vertex, degree in self.vertex_degree() if degree == 0]

    def density(self):
        """ returns the density of a graph """
        nbr_edges, nbr_vertices = float(len(self.edges())), float(len(self.vertices()))
        return 2 * nbr_edges / (nbr_vertices * (nbr_vertices - 1))

    def adjacency_matrix(self):
        """ returns the ajacency matrix of a graph
         in the form of a numpy array"""
        edges = self.edges()
        n = len(self.vertices())
        adj_matrix = [[0 for _ in range(n)] for _ in range(n)]
        for i in range(n):
            for j in range(n):
                if i != j and (self.vertices()[i], self.vertices()[j]) in edges:
                    adj_matrix[i][j], adj_matrix[j][i] = 1, 1
        return np.array(adj_matrix)

    def global_clustering_coeff(self):
        """ returns the global clustering coefficient of a graph """
        adj_mtrx = self.adjacency_matrix()
        path_length_two = np.linalg.matrix_power(adj_mtrx, 2)
        closed_triple_mtrx = np.multiply(adj_mtrx, path_length_two)
        n = len(self.vertices())
        nbr_closed_triple, nbr_all_triple = 0.0, 0.0  # float because of division
        nbr_closed_triple += sum(closed_triple_mtrx[i][e] for i in range(n) for e in range(n) if i != e)
        nbr_all_triple += sum(path_length_two[i][e] for i in range(n) for e in range(n) if i != e)
        # instead of not computing the diagonal
        # we could have substract np.trace(path_length_two) from nbr_triple
        return nbr_closed_triple / nbr_all_triple if nbr_all_triple != 0 else 0  # avoid 0 division

    def shortest_path(self, a, b):
        """ returns the shortest path distance between two given vertices a, b"""
        queue = deque()
        distance = {a: 0}
        queue.append(a)
        while len(queue) > 0:
            current = queue.popleft()
            for vertex in self.__graph_dict[current]:
                if vertex == b:
                    return distance[current] + 1
                if vertex not in distance:
                    queue.append(vertex)
                    distance[vertex] = distance[current] + 1
        return float("inf")

    def connected_components(self):
        """ returns a list of sets composed of two elements: the vertices list and the size
        of each connected components of a graph """
        components = []
        set_vertices = set(self.vertices())
        queue = deque()
        while len(set_vertices) > 0:
            init = set_vertices.pop()
            visited = {init: True}
            queue.append(init)
            while len(queue) > 0:
                current = queue.popleft()
                for vertex in self.__graph_dict[current]:
                    if vertex not in visited:
                        queue.append(vertex)
                    visited[vertex] = True
            set_vertices -= set(visited.keys())
            components.append(list(visited.keys()))
        return zip(components, map(lambda e: len(e), components))

    def connected_component_elements(self):
        """ returns a list of the vertices list of each connected components of a graph """
        return map(lambda x: x[0], self.connected_components())

    def component_diameter(self, component):
        """ returns the diameter of a given connected component element of a graph"""
        diameter = 0
        for init in component:
            queue = deque()
            distance = {init: 0}
            queue.append(init)
            while len(queue) > 0:
                current = queue.popleft()
                for vertex in self.__graph_dict[current]:
                    if vertex not in distance:
                        queue.append(vertex)
                        distance[vertex] = distance[current] + 1
            diameter = max((diameter, max(distance.values())))

        return diameter

    def forest_diameters(self):
        """ returns the list of the diameter of each connected components of a graph """
        return [self.component_diameter(component) for component in self.connected_component_elements()]

    def biggest_component_diameter(self):
        """ returns the diameter of the biggest connected component of a graph """
        return self.component_diameter(max(self.connected_component_elements(), key=len))

    def component_spanning_tree(self, component):
        """ returns the spanning tree of a given connected component of a graph """
        spanning_tree = Graph({})
        queue = deque()
        spanning_tree.add_vertex(component.pop())
        queue.extend(spanning_tree.vertices())
        while len(queue) > 0:
            current = queue.popleft()
            for vertex in self.__graph_dict[current]:
                if vertex not in spanning_tree.vertices():
                    queue.append(vertex)
                    spanning_tree.add_vertex(vertex)
                    spanning_tree.add_edge((current, vertex))
        return spanning_tree

    def spanning_forest(self):
        """ returns the list of spanning trees of each connected component of a graph """
        return [self.component_spanning_tree(component) for component in self.connected_component_elements()]

    def biggest_component_spanning_tree(self):
        """ returns the spanning tree of a the biggest connected component of a graph """
        return self.component_spanning_tree(max(self.connected_component_elements(), key=lambda c: len(c)))

    def __str__(self):
        """ A better way for printing a graph """
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
            res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res


## 1.1 Simple methods to get the ball rolling

In [3]:
G = {
    "a": ["c", "d", "g"],
    "b": ["c", "f"],
    "c": ["a", "b", "d", "f"],
    "d": ["a", "c", "e", "g"],
    "e": ["d"],
    "f": ["b", "c"],
    "g": ["a", "d"]
}
graph = Graph(G)
graph.add_vertex("h")
print("\n Vertices of graph G:")
print(graph.vertices())
print("\n Edges of graph G:")
print(graph.edges())


 Vertices of graph G:
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

 Edges of graph G:
[('a', 'c'), ('a', 'd'), ('a', 'g'), ('b', 'c'), ('b', 'f'), ('c', 'd'), ('c', 'f'), ('d', 'e'), ('d', 'g')]


## 2.1 Degree and isolated vertices

In [4]:
print("\n Degrees of graph G:")
print(graph.vertex_degree())
print("\n Isolated vertices of graph G:")
print(graph.find_isolated_vertices())


 Degrees of graph G:
[('a', 3), ('b', 2), ('c', 4), ('d', 4), ('e', 1), ('f', 2), ('g', 2), ('h', 0)]

 Isolated vertices of graph G:
['h']


## 2.2 Density calculation

In [5]:
def random_graph(n, p):
    """ create and returns a Erdos-Renyi
    G_n,p random graph -
    where n is the number of vertices
    and p the probability of puting
    an edge between each two vertices
     """
    graph = Graph({})
    for vertex in range(n):
        graph.add_vertex(vertex)
    for in_vertex in range(n):
        for out_vertex in range(n):
            if in_vertex < out_vertex:
                if random() < p:
                    graph.add_edge((in_vertex, out_vertex))
    return graph

print("\n Density of graph G:")
print(graph.density())

print("\n Density of an empty graph:")
empty_graph = random_graph(50, 0.)
print(empty_graph.density())

print("\n Density of a complete graph:")
complete_graph = random_graph(50, 1.)
print(complete_graph.density())

print("\n Density of a random graph R (p=0.05)")
random_graph = random_graph(50, 0.05)
print(random_graph.density())


 Density of graph G:
0.321428571429

 Density of an empty graph:
0.0

 Density of a complete graph:
1.0

 Density of a random graph R (p=0.05)
0.0391836734694


## 2.3 Degree sequence

In [6]:
print("\n Degree sequence of graph G:")
print(graph.degree_sequence())
print("\n Degree sequence of an empty graph:")
print(empty_graph.degree_sequence())
print("\n Degree sequence of a complete graph:")
print(complete_graph.degree_sequence())
print("\n Degree sequence of random graph R (p=0.05):")
print(random_graph.degree_sequence())


 Degree sequence of graph G:
[4, 4, 3, 2, 2, 2, 1, 0]

 Degree sequence of an empty graph:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

 Degree sequence of a complete graph:
[49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49]

 Degree sequence of random graph R (p=0.05):
[5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


## 2.4 Erdös-Gallai theorem

In [7]:
print("\n Erdos-Gallai test of graph G:")
print(graph.erdos_gallai(graph.degree_sequence()))
print("\n Erdos-Gallai test of an empty graph:")
print(empty_graph.erdos_gallai(empty_graph.degree_sequence()))
print("\n Erdos-Gallai test of a complete graph:")
print(complete_graph.erdos_gallai(complete_graph.degree_sequence()))
print("\n Erdos-Gallai test of random graph R (p=0.05):")
print(random_graph.erdos_gallai(random_graph.degree_sequence()))


 Erdos-Gallai test of graph G:
True

 Erdos-Gallai test of an empty graph:
True

 Erdos-Gallai test of a complete graph:
True

 Erdos-Gallai test of random graph R (p=0.05):
True


## 2.5 Clustering coefficient

In [8]:
print("\n Global clustering coefficient of graph G:")
print(graph.global_clustering_coeff())
print("\n Global clustering coefficient of an empty graph:")
print(empty_graph.global_clustering_coeff())
print("\n Global clustering coefficient of a complete graph:")
print(complete_graph.global_clustering_coeff())
print("\n Global clustering coefficient of random graph R (p=0.05):")
print(random_graph.global_clustering_coeff())


 Global clustering coefficient of graph G:
0.5

 Global clustering coefficient of an empty graph:
0

 Global clustering coefficient of a complete graph:
1.0

 Global clustering coefficient of random graph R (p=0.05):
0.111111111111


## 3.1 Connected components

In [9]:
print("\n List of vertices and respective size for each connected components of graph G:")
g_connected_components_size = graph.connected_components()
print(g_connected_components_size)
print("\n Number of connected components of random graph G:")
print(len(g_connected_components_size))

print("\n \n List of vertices and respective size for each connected components of random graph R (p=0.05):")
rg_connected_components_size = random_graph.connected_components()
print(rg_connected_components_size)
print("\n Number of connected components of random graph R (p=0.05):")
print(len(rg_connected_components_size))


 List of vertices and respective size for each connected components of graph G:
[(['a', 'c', 'b', 'e', 'd', 'g', 'f'], 7), (['h'], 1)]

 Number of connected components of random graph G:
2

 
 List of vertices and respective size for each connected components of random graph R (p=0.05):
[([0], 1), ([1, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 48, 49], 39), ([32, 43, 2, 11, 23], 5), ([3], 1), ([20], 1), ([33], 1), ([41], 1), ([47], 1)]

 Number of connected components of random graph R (p=0.05):
8


## 3.2 Shortest path

In [10]:
g_connected_components = graph.connected_component_elements()
g_vertices = sample(sample([c for c in g_connected_components if len(c) > 1], 1)[0], 2)
print("\n Shortest path distance between two random vertices "
      "({} and {}) in the same component of graph G:".format(*g_vertices))
print(graph.shortest_path(*g_vertices))
g_vertices = [sample(pop, 1)[0] for pop in sample(g_connected_components, 2)]
print("\n Shortest path distance between two random vertices "
    "({} and {}) in the different components of random graph R (p=0.05):".format(*g_vertices))
print(graph.shortest_path(*g_vertices))

rg_connected_components = random_graph.connected_component_elements()
components_not_single = [c for c in rg_connected_components if len(c) > 1]
if len(components_not_single) > 0:
    vertices = sample(sample(components_not_single, 1)[0], 2)
    print("\n Shortest path distance between two random vertices "
          "({} and {}) in the same component of random graph R (p=0.05):".format(*vertices))
    print(random_graph.shortest_path(*vertices))
if len(rg_connected_components) > 1:
    vertices = [sample(pop, 1)[0] for pop in sample(rg_connected_components, 2)]
    print("\n Shortest path distance between two random vertices "
          "({} and {}) in the different components of random graph R (p=0.05):".format(*vertices))
    print(random_graph.shortest_path(*vertices))


 Shortest path distance between two random vertices (c and b) in the same component of graph G:
1

 Shortest path distance between two random vertices (e and h) in the different components of random graph R (p=0.05):
inf

 Shortest path distance between two random vertices (32 and 43) in the same component of random graph R (p=0.05):
1

 Shortest path distance between two random vertices (41 and 33) in the different components of random graph R (p=0.05):
inf


## 3.3 Diameter

In [11]:
print("\n Diameter for each component of graph G:")
print(graph.forest_diameters())
print("\n Diameter of the biggest component of graph G:")
print(graph.biggest_component_diameter())

print("\n \n Diameter for each component of random graph R (p=0.05):")
print(random_graph.forest_diameters())
print("\n Diameter of the biggest component of random graph R (p=0.05):")
print(random_graph.biggest_component_diameter())


 Diameter for each component of graph G:
[3, 0]

 Diameter of the biggest component of graph G:
3

 
 Diameter for each component of random graph R (p=0.05):
[0, 13, 4, 0, 0, 0, 0, 0]

 Diameter of the biggest component of random graph R (p=0.05):
13


## 3.4 Spanning tree

In [12]:
print("\n List of edges of the spanning tree, for each component of graph G:")
print([sp.edges() for sp in graph.spanning_forest()])
print("\n Number of edges of the spanning tree, for each component of graph G:")
print([len(sp.edges()) for sp in graph.spanning_forest()])
print("\n Number of vertices for each component of graph G:")
print([len(c) for c in g_connected_components])


print("\n \n List of edges of the spanning tree, for each component of random graph R (p=0.05):")
print([sp.edges() for sp in random_graph.spanning_forest()])
print("\n Number of edges of the spanning tree, for each component of random graph R (p=0.05):")
print([len(sp.edges()) for sp in random_graph.spanning_forest()])
print("\n Number of vertices for each component of random graph R (p=0.05):")
print([len(c) for c in rg_connected_components])


 List of edges of the spanning tree, for each component of graph G:
[[('a', 'c'), ('a', 'g'), ('b', 'f'), ('c', 'f'), ('c', 'd'), ('d', 'e')], []]

 Number of edges of the spanning tree, for each component of graph G:
[6, 0]

 Number of vertices for each component of graph G:
[7, 1]

 
 List of edges of the spanning tree, for each component of random graph R (p=0.05):
[[], [(1, 42), (4, 15), (4, 13), (5, 29), (6, 40), (6, 46), (7, 36), (7, 16), (8, 9), (9, 18), (10, 31), (10, 17), (12, 34), (12, 26), (13, 48), (14, 37), (15, 29), (16, 42), (17, 18), (19, 31), (19, 35), (21, 28), (22, 30), (22, 34), (24, 31), (25, 30), (25, 36), (25, 39), (27, 49), (28, 37), (28, 31), (29, 36), (30, 49), (35, 44), (36, 37), (37, 45), (38, 45), (39, 40)], [(2, 23), (2, 11), (23, 32), (32, 43)], [], [], [], [], []]

 Number of edges of the spanning tree, for each component of random graph R (p=0.05):
[0, 38, 4, 0, 0, 0, 0, 0]

 Number of vertices for each component of random graph R (p=0.05):
[1, 39, 5, 

## 4.1 Importing real data

In [13]:
# =========================
#      IMPORT DATA
# =========================


def file_to_graph(file_path):
    """ import and parse a text file containing an edge list
    then dynamically construct a dictionnary representation of the graph from the edge list"""
    graph_import = Graph({})
    with open(file_path, 'r') as document:
        for line in document:
            vertices = line.split()
            if not vertices:  # empty line?
                continue
            if len(vertices) % 2 == 0:
                graph_import.add_vertex(vertices[0])
                graph_import.add_vertex(vertices[1])
                graph_import.add_edge((vertices[0], vertices[1]))
            else:
                for v in vertices:
                    graph_import.add_vertex(v)
                    if v != vertices[0]:
                        graph_import.add_edge((vertices[0], v))
    return graph_import

file_list = ('zachary-connected.txt', 'graph_100n_1000m.txt', 'graph_1000n_4000m.txt')
for file_path in file_list:
    print("\n \n Graph constructed from {} :".format(file_path))
    graph = file_to_graph(file_path)
    print("\n Number of vertices :")
    print(len(graph.vertices()))
    print("\n Number of edges :")
    print(len(graph.edges()))
    print("\n Density:")
    print(graph.density())
    print("\n Diameter:")
    print(graph.biggest_component_diameter())
    print("\n Clustering coefficient:")
    print(graph.global_clustering_coeff())


 
 Graph constructed from zachary-connected.txt :

 Number of vertices :
33

 Number of edges :
78

 Density:
0.147727272727

 Diameter:
5

 Clustering coefficient:
0.154580152672

 
 Graph constructed from graph_100n_1000m.txt :

 Number of vertices :
100

 Number of edges :
960

 Density:
0.193939393939

 Diameter:
3

 Clustering coefficient:
0.192757201646

 
 Graph constructed from graph_1000n_4000m.txt :

 Number of vertices :
1000

 Number of edges :
3989

 Density:
0.00798598598599

 Diameter:
6

 Clustering coefficient:
0.00699785651242


## 4.2 Properties of supplied graphs

### Graph constructed from zachary-connected.txt :

 Number of vertices :
33

 Number of edges :
78

 Density:
0.147727272727

 Diameter:
5

 Clustering coefficient:
0.154580152672

 
### Graph constructed from graph_100n_1000m.txt :

 Number of vertices :
100

 Number of edges :
960

 Density:
0.193939393939

 Diameter:
3

 Clustering coefficient:
0.192757201646

 
### Graph constructed from graph_1000n_4000m.txt :

 Number of vertices :
1000

 Number of edges :
3989

 Density:
0.00798598598599

 Diameter:
6

 Clustering coefficient:
0.00699785651242